In [1]:
import datasets
from datasets import load_dataset

In [18]:
raw_datasets = load_dataset("kilt_tasks", "eli5")

Reusing dataset kilt_tasks (/u/myu2/.cache/huggingface/datasets/kilt_tasks/eli5/1.0.0/57dc8b2431e76637e0c6ef79689ca4af61ed3a330e2e0cd62c8971465a35db3a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
raw_datasets["train"].to_json("/dccstor/myu/data/kilt_eli5_tmp/eli5-train-kilt.json")

Creating json from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

599038772

In [24]:
examples = load_dataset("json", data_files={
    "validation": "/dccstor/myu/data/kilt_eli5/eli5-dev-kilt.json"
}, split="validation")

Using custom data configuration default-97f3a9677bc7fc92
Reusing dataset json (/u/myu2/.cache/huggingface/datasets/json/default-97f3a9677bc7fc92/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


In [28]:
references = [{"id": ex["id"], "answers": [x["answer"] for x in ex["output"] if x["answer"] is not None]} for ex in examples] # muli references


In [29]:
examples[0]

{'id': '1kiwfx',
 'input': 'In Trading Places (1983, Akroyd/Murphy) how does the scheme at the end of the movie work? Why would buying a lot of OJ at a high price ruin the Duke Brothers?',
 'output': [{'answer': 'The final scene involves future contracts. This simply means entering into a contract to buy something (oil, wheat, even frozen concentrated orange juice(FCOJ)) at a specified time for the current price. The person selling the future does not have to own the FCOJ at the time of sale he simply has to provide them at the agreed upon date. Futures help companies mitigate risk against the unpredictable price of FCOJ. If the price of FCOJ goes up the buyer wins the seller loses and visa versa. This price is often affected by fresh oranges. If there is a good harvest FCOJ price goes down and so on. The Dukes believed there was going to be a bad harvest. Their plan was to buy as much FCOJ as they could and basically corner the market then sell it at a much higher price due to a lack 

In [30]:
references[0]

{'id': '1kiwfx',
 'answers': ['The final scene involves future contracts. This simply means entering into a contract to buy something (oil, wheat, even frozen concentrated orange juice(FCOJ)) at a specified time for the current price. The person selling the future does not have to own the FCOJ at the time of sale he simply has to provide them at the agreed upon date. Futures help companies mitigate risk against the unpredictable price of FCOJ. If the price of FCOJ goes up the buyer wins the seller loses and visa versa. This price is often affected by fresh oranges. If there is a good harvest FCOJ price goes down and so on. The Dukes believed there was going to be a bad harvest. Their plan was to buy as much FCOJ as they could and basically corner the market then sell it at a much higher price due to a lack of oranges. So here is what happened. At first Winthrop and Valentine begin selling futures contracts at inflated prices caused by the Dukes (on the info from the fake report of a ba

In [3]:

import transformers
from typer import Option
from oneqa.lfqa_kb.trainers.seq2seq_trainer import QuestionAnsweringSeq2SeqTrainer
from oneqa.lfqa_kb.metrics.utils import compute_metrics
from oneqa.lfqa_kb.models.fid import FiDBART
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    set_seed,
    BartForConditionalGeneration,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version


In [2]:
bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
model = FiDBART(bart.config)
model.load_pretrained(bart.state_dict())

In [18]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained('facebook/bart-base')
tokenizer.pad_token = tokenizer.unk_token

features = [
  {"input_ids": [[1,2,3],[4,5,6]], "attention_mask": [[1,1,1],[1,1,0]], "labels": [-100,-100,-100,-100,-100,6]},
  {"input_ids": [[1,2,3], [4,4,4]], "attention_mask": [[1,1,0],[1,1,1]], "labels": [-100,-100,4]},
]
output = transformers.DataCollatorForSeq2Seq(tokenizer)(features)

x = output['attention_mask'][0, :].tolist()
print(x) # [1, 1, 1, 0, 0, 0]
assert x == [1,1,0] + [0] * 3

[[1, 1, 1], [1, 1, 0]]


AssertionError: 

In [27]:
features = [{'input_ids': [[0, 40018, 35, 1336, 109, 52, 216, 70, 5, 418, 5, 168, 16, 562, 31, 827, 13070, 16, 164, 124, 7, 5, 82, 116, 9078, 35, 96, 3454, 6, 10, 2228, 8068, 36, 368, 2228, 1361, 43, 16, 10, 8068, 9, 418, 30, 10, 582, 254, 2024, 88, 10, 582, 1942, 18, 827, 1316, 4, 10480, 8874, 32, 144, 10266, 156, 30, 1252, 11, 5, 3207, 9, 9241, 8, 6729, 8, 13, 5, 3207, 9, 7593, 108, 2349, 6, 53, 5, 2122, 64, 28, 341, 13, 3081, 13, 143, 3508, 6, 215, 25, 3207, 9, 4033, 6, 2556, 6, 8, 97, 168, 1103, 4, 10480, 8874, 32, 144, 10266, 156, 30, 839, 9, 5175, 1188, 12291, 36689, 634, 804, 6, 1830, 6, 8, 7377, 3454, 1743, 53, 2], [0, 40018, 35, 1336, 109, 52, 216, 70, 5, 418, 5, 168, 16, 562, 31, 827, 13070, 16, 164, 124, 7, 5, 82, 116, 9078, 35, 6389, 3454, 16, 10, 5177, 32598, 18670, 23920, 613, 518, 32, 16556, 8094, 1195, 87, 30574, 11, 10, 4828, 827, 4, 6389, 3454, 34, 63, 8121, 11, 8178, 13426, 8, 21, 144, 682, 5091, 13, 304, 30, 5177, 12, 506, 13598, 3353, 36, 41583, 6209, 43, 25, 10, 169, 7, 797, 1042, 4, 8451, 256, 14071, 3375, 3454, 6448, 58, 18163, 1070, 30, 2974, 4344, 225, 788, 8, 172, 423, 2226, 30, 1134, 215, 25, 20381, 4054, 1016, 3787, 610, 19802, 4, 3687, 382, 12, 805, 786, 12, 7699, 2244, 89, 32, 23, 513, 1105, 5177, 506, 13598, 3353, 36, 41583, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},{'input_ids': [[0, 40018, 35, 1336, 109, 52, 216, 70, 5, 418, 5, 168, 16, 562, 31, 827, 13070, 16, 164, 124, 7, 5, 82, 116, 9078, 35, 96, 3454, 6, 10, 2228, 8068, 36, 368, 2228, 1361, 43, 16, 10, 8068, 9, 418, 30, 10, 582, 254, 2024, 88, 10, 582, 1942, 18, 827, 1316, 4, 10480, 8874, 32, 144, 10266, 156, 30, 1252, 11, 5, 3207, 9, 9241, 8, 6729, 8, 13, 5, 3207, 9, 7593, 108, 2349, 6, 53, 5, 2122, 64, 28, 341, 13, 3081, 13, 143, 3508, 6, 215, 25, 3207, 9, 4033, 6, 2556, 6, 8, 97, 168, 1103, 4, 10480, 8874, 32, 144, 10266, 156, 30, 839, 9, 5175, 1188, 12291, 36689, 634, 804, 6, 1830, 6, 8, 7377, 3454, 1743, 53, 2], [0, 40018, 35, 1336, 109, 52, 216, 70, 5, 418, 5, 168, 16, 562, 31, 827, 13070, 16, 164, 124, 7, 5, 82, 116, 9078, 35, 6389, 3454, 16, 10, 5177, 32598, 18670, 23920, 613, 518, 32, 16556, 8094, 1195, 87, 30574, 11, 10, 4828, 827, 4, 6389, 3454, 34, 63, 8121, 11, 8178, 13426, 8, 21, 144, 682, 5091, 13, 304, 30, 5177, 12, 506, 13598, 3353, 36, 41583, 6209, 43, 25, 10, 169, 7, 797, 1042, 4, 8451, 256, 14071, 3375, 3454, 6448, 58, 18163, 1070, 30, 2974, 4344, 225, 788, 8, 172, 423, 2226, 30, 1134, 215, 25, 20381, 4054, 1016, 3787, 610, 19802, 4, 3687, 382, 12, 805, 786, 12, 7699, 2244, 89, 32, 23, 513, 1105, 5177, 506, 13598, 3353, 36, 41583, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}]

In [33]:
batch_features = {}
for feature in features:
    for k,v in feature.items(): 
        if k not in batch_features:
            batch_features[k] = []
        batch_features[k].append(v)
for k,v in batch_features.items():
    batch_features[k] = torch.tensor(v) # convert to tensor

# prepare decoder_input_ids
    print(batch_features["input_ids"])

tensor([[[    0, 40018,    35,  1336,   109,    52,   216,    70,     5,   418,
              5,   168,    16,   562,    31,   827, 13070,    16,   164,   124,
              7,     5,    82,   116,  9078,    35,    96,  3454,     6,    10,
           2228,  8068,    36,   368,  2228,  1361,    43,    16,    10,  8068,
              9,   418,    30,    10,   582,   254,  2024,    88,    10,   582,
           1942,    18,   827,  1316,     4, 10480,  8874,    32,   144, 10266,
            156,    30,  1252,    11,     5,  3207,     9,  9241,     8,  6729,
              8,    13,     5,  3207,     9,  7593,   108,  2349,     6,    53,
              5,  2122,    64,    28,   341,    13,  3081,    13,   143,  3508,
              6,   215,    25,  3207,     9,  4033,     6,  2556,     6,     8,
             97,   168,  1103,     4, 10480,  8874,    32,   144, 10266,   156,
             30,   839,     9,  5175,  1188, 12291, 36689,   634,   804,     6,
           1830,     6,     8,  7377,  3

In [31]:
output_features = tokenizer.pad(features, padding="max_length", max_length=6, return_tensors='pt')
print(output_features["input_ids"].shape)

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [ ]:
raw_datasets["validation"][0]["output"]

In [5]:
p = tokenizer(["I am a big fan of IBM."])

In [6]:
p["input_ids"]

[[0, 100, 524, 10, 380, 2378, 9, 11510, 4, 2]]

In [12]:
import torch
a = torch.tensor(p["attention_mask"])

In [14]:
a.bool()

tensor([[True, True, True, True, True, True, True, True, True, True]])

In [15]:
output[2]

{'answer': 'In most cases the O-Line is supposed to make a hole for the running back to go through. If you run too many plays to the outside/throws the defense will catch on.\n\nAlso, 2 5 yard plays gets you a new set of downs.',
 'meta': {'score': 2},
 'provenance': []}

In [6]:
import json
from collections import Counter
ans_score_list = []
with open ("/dccstor/myu/OneQA/oneqa/lfqa_kb/data_models/kilt_eli5/eli5-train-kilt.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line.strip())
        answer_list = data["output"]
        ans_score_cnt = Counter()
        if len(answer_list) > 0:
            for ans in answer_list:
                ans_text = ans["answer"]
                ans_score = ans["meta"]["score"]
                ans_score_cnt[ans_score] += 1
        ans_score_list.append(ans_score_cnt)
                

how many instances have at least one 3 score:

In [28]:
tmp = 0
highest_score_cnt = Counter()
for cnt in ans_score_list:
    answer_num = sum([v for k,v in cnt.items()])
    scores = [k for k,v in cnt.items()]
    highest_score = max(scores)
    if highest_score >= 3:
        tmp += 1
    else:
        if answer_num == 1:
            tmp += 1
        highest_score_cnt[max(scores)] += 1
        

            
        
print(tmp)

259498


In [27]:
for k,v in highest_score_cnt.items():
    print(k,v)

2 52479


In [8]:
len(ans_score_list)

272634

In [14]:
3 in ans_score_list[0]

True

In [5]:
from datasets import load_metric
metric = load_metric("rouge")
refs = [{      "id": "1qdjpv",
        "answers": [
            "There probably would be similarities. Things that were swimmers would probably be sleek, for instance, due to natural selection. It's just that there would have been an entirely different evolutionary history and so different things may/would have been tried that didn't get a chance on our world.",
            "There are many ways that human race could have turned out - the problems we needed to evolve for would, most of the time, have more than one solution. That means chances are that life on a similar planet to ours would have solved the problems in a different way - Maybe giving us more blubber and less hair to combat cold; more opposable thumbs; more eyes, etc etc. Also, if other things on the world are a little different it could cause an evolutionary butterfly effect, making them much different to us. Another factor would be if there was an unlucky disaster that wiped out all of the species similar to ours maybe their version of the Neanderthal lived on and became the dominant species. So many variables = a species with some similarities to us but probably much different",
            "Life evolves through random mutations. This image demonstrates how you could take one of two paths, and keep doing this over and over until your destination is incredibly distant.  URL_0  Life doesn't have to live in the same way we do. There are the radioactive fungi recently discovered for example. We just live by one set of rules and there's no reason why we couldn't have lived by another.",
            "I guess the first question is, what do you mean by habitable? Same atmosphere? Same atmosphere earth started with, or same atmosphere earth has now? Let's do a thought experiment. We'll take a bunch of different planets that started out identical to earth, but \"split off\" at different points. So, that would be like saying \"what if I hadn't gotten on the train last tuesday\". Your life might be different, you might have lost your job, or found your wife, but humanity would not cease to exist (nor would you not be born). You get the idea. First stop, a planet that only branched 5 million years ago. If you visited this planet, you will still recognize many animals, or at least animal types. Humanity may or may not have ever evolved (at least to the point you see them now) but other than that, the earth would look quite similar. Next we'll branch at 200 million years ago. Maybe now the planet you're coming to (in the present) still has dinosaurs, maybe mammals never took hold. No matter how evolution continued in this world, you may recognize very few TYPES of animals, but you'll still recongize them AS animals. Insects will be relatively recognizable, plants will as well. Next we'll branch at 550 MYA. At this point, it becomes more of a crapshoot. Depending on the circumstances, land plants might not have evolved. Sure, something would fill that niche, but that something might look entirely foreign to you. We're also missing vertebrates (all higher animals). Again, something would probably evolve to fill that niche, but that something would likely not have the body plan that's so familiar to us. The world would look increasingly alien. Of course, again, we could still recognize what lived there as life, and there would still be animals, and fungus, and all sorts of things we could at least SOMEWHAT relate to. Now for the gist of your question, what if the planets branched before life even evolved? That's where we really don't know, but we can speculate. The biggest question is, is the basic structure of our DNA a chemical necessity (as in, there is no other way to have a self-replicating molecule to build higher life on), or was it a fluke? Are there a variety of different molecules that could form, and could self replicate in the way DNA/RNA did, and the only reason why we have DNA is because once SOMETHING like that forms, it will immediately become the ONLY thing, and change the environment in such a way to prevent any other \"things\" from ever forming? If that's the case, a planet that started out like earth could very well end up with \"life\" that is so alien we might not even recognize it AS life.",
            "it would be less of a question of *are they simalir* and more of a question *how far along in the natural history are we on that planet, and how are specific traits exhibited, and are there any traits that we have never seen before?* you will have sleek swimmers, maybe they have fins. maybe they have jet propulsion, maybe they just float there. the major fluid present is a factor here. you will have flyers. how many wings, or do they use buoyancy? air pressure becomes a factor here, as well as gravity, and atmospheric components. they will have *mouths* or some way to eat/gain nutrition. but how, would they have a standard digestive system? maybe more of a venom based system is selected for, leading to a whole planet looking like Australia. would the plants ue sunlight, or be more like the fungi from earths early history? perhaps hey get selected more towards carnivorous plants, and you have things like giant Venus flytraps, or pitcher plants more common then here on earth. maybe they use a different system entirely, and it might be totally possible for a plant and animal symbioid to appear *think bulbasaur* youll have things with legs, youll have prehensile limbs, youll have horns or a means of self defense. there will be a anolog for all of these things, its just how they are expressed that will be diffrent. and weather or not life on that planet has been around long enough for its life to get a chance to express analogs to our life. animals will have some sort of camoflauge, we dont know what kind exactly, but we can guess it probably would depend on the local environment and selective pressure. but it will be there in SOME form. and if the planet has had life long enough MAYBE it might have a intelligent species. now...would that be a hive mind caste system like ants? like us? are they aquatic like dolphins or octopi? do they communicate with voice? or by pheromones? maybe they comunicate by flashes of light? eyes will evolve unless its a very dark world. but where would they be? what would they look like? maybe compound style eyes are preferred. maybe the use their whole body to see. another thing to consider, if you had visited earth a very long time ago, you wouldn't see much of anything that looks like today animals. giant bugs existed cause oxygen was so plentiful in the atmosphere. hell, go deep enough into the sea and you can find fish that can stretch their bellies like 5 times their size. and ones that entrance their prey with light. we have electric eels. ELECTRIC. FUCKING. EELS. if you have never heard of them, they delver a sizable shock of around 600 fucking volts. idr what amperage its at. but, hell i didn't think that was possible till i heard of it. and that's on EARTH. point is, we wont know till we look. we can be sure the same niches will exist, its just how the alien life solves it would be the question.",
            ""
        ]}]
preds =  [{
        "id": "1qdjpv",
        "prediction_text": "There probably would be similarities. Things that were swimmers would probably be sleek, for instance, due to natural selection. It's just that there would have been an entirely different evolutionary history and so different things may/would have been tried that didn't get a chance on our world.",

    }],
metric.compute(predictions=preds, references=refs)

{'rouge1': AggregateScore(low=Score(precision=0.9629629629629629, recall=0.040912667191188044, fmeasure=0.07849056603773585), mid=Score(precision=0.9629629629629629, recall=0.040912667191188044, fmeasure=0.07849056603773585), high=Score(precision=0.9629629629629629, recall=0.040912667191188044, fmeasure=0.07849056603773585)),
 'rouge2': AggregateScore(low=Score(precision=0.9433962264150944, recall=0.03937007874015748, fmeasure=0.07558578987150416), mid=Score(precision=0.9433962264150944, recall=0.03937007874015748, fmeasure=0.07558578987150416), high=Score(precision=0.9433962264150944, recall=0.03937007874015748, fmeasure=0.07558578987150416)),
 'rougeL': AggregateScore(low=Score(precision=0.9629629629629629, recall=0.040912667191188044, fmeasure=0.07849056603773585), mid=Score(precision=0.9629629629629629, recall=0.040912667191188044, fmeasure=0.07849056603773585), high=Score(precision=0.9629629629629629, recall=0.040912667191188044, fmeasure=0.07849056603773585)),
 'rougeLsum': Aggre